
# 基于修改后的 MindQuantum 0.5.0 采用变分算法重构半导体双量子点单-三重态量子 CX 门

# 采用自定义的 MQLayer abs 来实现

In [9]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/CNOT_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/CNOT_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/CNOT_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/CNOT_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

infid_list = [] # mean of infid
error_list = [] # max of infid
lr = 0.1

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('mqvector', circ.n_qubits)
sim_left = Simulator('mqvector',circ.n_qubits)

grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
m = 0 # 用于计数 
fid_max = 0
fid_eval_max = 0
for j in range(len(train_x)):
    res = net(Tensor(train_x[j]), Tensor(train_y[j]))
    if res[0,0] > fid_max or j%100==0:
        fid_max = max(fid_max, res[0,0])
        print('\nfid_max:', fid_max, 'i:', j)
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        fid_eval = np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
        fid_eval_max = max(fid_eval_max, fid_eval)
        print('fid_eval_max:',fid_eval_max, 'fid_eval:', fid_eval)
        
# fid_max: 0.999902 i: 2157
# fid_eval_max: 0.9994103731512852 fid_eval: 0.9991309956328116


fid_max: 0.5161779 i: 0
fid_eval_max: 0.04705876975848316 fid_eval: 0.04705876975848316

fid_max: 0.6141951 i: 3
fid_eval_max: 0.08969107733524914 fid_eval: 0.08969107733524914

fid_max: 0.6474364 i: 5
fid_eval_max: 0.08969107733524914 fid_eval: 0.08553935298495903

fid_max: 0.8532114 i: 10
fid_eval_max: 0.1686321335094972 fid_eval: 0.1686321335094972

fid_max: 0.88086706 i: 23
fid_eval_max: 0.6809218204786021 fid_eval: 0.6809218204786021

fid_max: 0.90866256 i: 46
fid_eval_max: 0.6997058894599907 fid_eval: 0.6997058894599907

fid_max: 0.90866256 i: 100
fid_eval_max: 0.7143077819435213 fid_eval: 0.7143077819435213

fid_max: 0.92959166 i: 117
fid_eval_max: 0.8778321296272341 fid_eval: 0.8778321296272341

fid_max: 0.93221647 i: 126
fid_eval_max: 0.8778321296272341 fid_eval: 0.8622968079848766

fid_max: 0.9883202 i: 145
fid_eval_max: 0.8778321296272341 fid_eval: 0.8372576453429879

fid_max: 0.9883202 i: 200
fid_eval_max: 0.8778321296272341 fid_eval: 0.6739914481047691

fid_max: 0.9883202

In [3]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cx_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cx_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cx_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cx_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('mqvector', circ.n_qubits)
sim_left = Simulator('mqvector',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.05
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    if error < error_min:
        error_min = error
        params_tem = params
        j_tem = j
    if j % 50 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-5:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('params_tem:\n', params_tem)

error_min: 0.9344227390500963 error_now: 0.9344227390500963 train_num: 0
error_min: 0.18707726294315064 error_now: 0.2224786599881965 train_num: 50
error_min: 0.09122507669191227 error_now: 0.09991302978380401 train_num: 100
error_min: 0.09122507669191227 error_now: 0.21148050805691232 train_num: 150
error_min: 0.09122507669191227 error_now: 0.2661200669849586 train_num: 200
error_min: 0.060363093454498085 error_now: 0.06975054471592068 train_num: 250
error_min: 0.060363093454498085 error_now: 0.07814969942078154 train_num: 300
error_min: 0.04797640778561907 error_now: 0.11732427286745106 train_num: 350
error_min: 0.04797640778561907 error_now: 0.08709595845186013 train_num: 400
error_min: 0.04797640778561907 error_now: 0.06702499341321333 train_num: 450
error_min: 0.04797640778561907 error_now: 0.08796698402188474 train_num: 500
error_min: 0.04797640778561907 error_now: 0.25978454278764296 train_num: 550
error_min: 0.04797640778561907 error_now: 0.1279056719458066 train_num: 600
error

error_min: 0.04797640778561907 error_now: 0.2610912950188613 train_num: 5300
error_min: 0.04797640778561907 error_now: 0.2574529416795339 train_num: 5350
error_min: 0.04797640778561907 error_now: 0.2712633633372684 train_num: 5400
error_min: 0.04797640778561907 error_now: 0.15755535597223969 train_num: 5450
error_min: 0.04797640778561907 error_now: 0.16682371195460544 train_num: 5500
error_min: 0.04797640778561907 error_now: 0.166188621450158 train_num: 5550
error_min: 0.04797640778561907 error_now: 0.17302744399830206 train_num: 5600
error_min: 0.04797640778561907 error_now: 0.18163728369561583 train_num: 5650
error_min: 0.04797640778561907 error_now: 0.16544330905202764 train_num: 5700
error_min: 0.04797640778561907 error_now: 0.18292586732290594 train_num: 5750
error_min: 0.04797640778561907 error_now: 0.17336322211955257 train_num: 5800
error_min: 0.04797640778561907 error_now: 0.1294092035890113 train_num: 5850
error_min: 0.04797640778561907 error_now: 0.49980756152186434 train_nu

In [1]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cx_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cx_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cx_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cx_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('mqvector', circ.n_qubits)
sim_left = Simulator('mqvector',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.01
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    if error < error_min:
        error_min = error
        params_tem = params
        j_tem = j
    if j % 50 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-5:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('params_tem:\n', params_tem)

error_min: 0.9441378783013046 error_now: 0.9441378783013046 train_num: 0
error_min: 0.783312709481143 error_now: 0.783312709481143 train_num: 50
error_min: 0.08822126361334559 error_now: 0.08822126361334559 train_num: 100
error_min: 0.03308202264383642 error_now: 0.03684451164030966 train_num: 150
error_min: 0.03308202264383642 error_now: 0.04944551869846869 train_num: 200
error_min: 0.03308202264383642 error_now: 0.06938714600419116 train_num: 250
error_min: 0.03308202264383642 error_now: 0.048101990076230394 train_num: 300
error_min: 0.03308202264383642 error_now: 0.042143790431671824 train_num: 350
error_min: 0.02973382556355164 error_now: 0.035892231494172755 train_num: 400
error_min: 0.02973382556355164 error_now: 0.030352244540728823 train_num: 450
error_min: 0.029488509496666593 error_now: 0.04442460177027041 train_num: 500
error_min: 0.029488509496666593 error_now: 0.046522714342020266 train_num: 550
error_min: 0.029488509496666593 error_now: 0.046006425889988156 train_num: 600